In [1]:
def fc(x, scope, nh, *, init_scale=1.0, init_bias=0.0):
    with tf.variable_scope(scope):
        nin = x.get_shape()[1].value
        w = tf.get_variable("w", [nin, nh])
        b = tf.get_variable("b", [nh])
        return (tf.matmul(x, w)+b)


In [2]:
import tensorflow as tf;
import sklearn.datasets;
iris_ds = sklearn.datasets.load_iris(return_X_y=False)
x = tf.placeholder(tf.float32, shape=(10, 4))
y = tf.placeholder(tf.float32, shape=(10,3))

In [3]:
#Printing the dataset
print (iris_ds["data"].shape)
print (iris_ds["target"].shape)
print (iris_ds["target"])

(150, 4)
(150,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [4]:
from random import shuffle
import numpy as np;
iris_data = np.array(iris_ds["data"])
iris_label = np.array(iris_ds["target"])
length = len(iris_label)
iris_idx = list(range(length))
shuffle(iris_idx)
train_idx = iris_idx[:round(length*2/3)]
test_idx = iris_idx[round(length*2/3):]

train_data,train_label = iris_data[train_idx,:],iris_label[train_idx]
test_data,test_label = iris_data[test_idx,:],iris_label[test_idx]
print (len(train_data))
#Reshaping Dataset
train_data,train_label = train_data.reshape(10,-1,4),train_label.reshape(10,-1)
print (train_data.shape,train_label.shape)

100
(10, 10, 4) (10, 10)


In [5]:
def build_model(x):
    y1 = tf.nn.relu(fc(x,nh=5,scope="fc1"))
    y2 = fc(y1,nh=3,scope = "fc2")
    return tf.nn.softmax(y2)

y_hat = build_model(x)

entropy_loss = tf.losses.softmax_cross_entropy(logits=y_hat,onehot_labels=y)
#For Calculating Accuracy

In [6]:
correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y, 1))
# Calculate accuracy for 3000 examples
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
#Optimizers
optimizer = tf.train.AdamOptimizer().minimize(entropy_loss)

In [11]:
train_data = train_data/np.linalg.norm(train_data)

with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    #Training the model
    for epoch in range(3000):
        for (dt,lbl) in zip(train_data,train_label):
            #print (dt,tf.keras.utils.to_categorical(lbl,3))
            _acc,_loss,opt= sess.run([accuracy,entropy_loss,optimizer],feed_dict={x:dt,y:tf.keras.utils.to_categorical(lbl,3)})
        if (epoch%10==0):
            print (_acc,_loss)

0.4 1.0477346
0.4 1.0474279
0.4 1.0474671
0.4 1.0477847
0.4 1.048301
0.5 1.0489582
0.5 1.0497135
0.5 1.0505339
0.5 1.0513939
0.5 1.052273
0.5 1.0531553
0.5 1.0540272
0.5 1.0548778
0.5 1.0556984
0.5 1.0564823
0.5 1.057225
0.5 1.0579218
0.5 1.0585674
0.5 1.0591532
0.9 1.0596629
0.7 1.0600713
0.4 1.0603423
0.5 1.0604284
0.5 1.0602741
0.5 1.0598235
0.5 1.0590256
0.5 1.0578421
0.5 1.0562524
0.5 1.0542531
0.5 1.0518576
0.5 1.0490917
0.5 1.0459898
0.5 1.0425904
0.5 1.038935
0.5 1.0350631
0.5 1.0310137
0.5 1.0268207
0.5 1.0225142
0.5 1.0181193
0.5 1.0136557
0.5 1.0091388
0.5 1.0045786
0.5 0.9999823
0.5 0.9953535
0.5 0.99069387
0.5 0.98600334
0.5 0.9812813
0.5 0.9765272
0.5 0.97174114
0.5 0.9669239
0.5 0.96207756
0.5 0.95720524
0.5 0.952311
0.5 0.9474001
0.5 0.9424782
0.5 0.93755066
0.5 0.9326237
0.5 0.92770255
0.5 0.9227921
0.5 0.91789657
0.6 0.91301966
0.6 0.9081645
0.8 0.90333337
0.8 0.8985285
0.8 0.89375144
0.8 0.88900346
0.9 0.8842856
0.9 0.8795991
0.9 0.8749445
0.9 0.870323
0.9 0.86573535

In [12]:
logs_path = 'C:\\Users\\jaley\\git\\sess2'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
